In [1]:
from scipy.io import savemat

import pandas as pd
import numpy as np
import pickle as pkl
from sklearn.preprocessing import StandardScaler

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt

import tqdm
import seaborn as sns

from models import MLP, TandemNet, cVAE, cGAN, INN
from utils import evaluate_simple_inverse, evaluate_tandem_accuracy, evaluate_vae_inverse, evaluate_gan_inverse, evaluate_inn_inverse
from configs import get_configs
from plotting_utils import compare_cie_dist, compare_param_dist, plot_cie, plot_cie_raw_pred
from datasets import get_dataloaders, SiliconColor

from sklearn.metrics import r2_score

#DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
DEVICE = 'cpu'
train_loader, val_loader, test_loader = get_dataloaders('tandem_net')


### Define function for checking the correctness of predicted structures

In [2]:
def struc_check(structure):
    if np.sum(abs(structure)-structure)>0:  # if get negative parameters, then wrong structure
        return 0
    else:
        struc = np.reshape(structure, (-1, 4));
        N = np.shape(struc)[0]
        #print(struc)
        for i in range(N):
            if (struc[i,1]+struc[i,3]>=struc[i,2]):  # if gap+diameter >= period, then wrong structure
                return 0;
            
        return 1;
    
def struc_remove(cie_raw, param_raw, param_pred):
    # remove all structures predicted that is not satisfied by struc_check
    M = np.shape(cie_raw)[0]
    j = 0
    B = []
    for i in range(M-1):
        if struc_check(param_pred[i,:])==0:
            param_pred[j,:] = param_pred[i+1,:]
            param_raw[j,:] = param_raw[i+1,:]
            cie_raw[j,:] = cie_raw[i+1,:]
            B.append(i)
            print(i,j)
        else:
            param_pred[j,:] = param_pred[i,:]
            param_raw[j,:] = param_raw[i,:]
            cie_raw[j,:] = cie_raw[i,:]
            j = j+1
    i = i+1
    
    if struc_check(param_pred[i,:])==0:
        param_pred = param_pred[0:j,:]
        param_raw = param_raw[0:j,:]
        cie_raw = cie_raw[0:j,:]
    else:
        param_pred[j,:] = param_pred[i,:]
        param_raw[j,:] = param_raw[i,:]
        cie_raw[j,:] = cie_raw[i,:]
        param_pred = param_pred[0:(j+1),:]
        param_raw = param_raw[0:(j+1),:]
        cie_raw = cie_raw[0:(j+1),:]

    return B, cie_raw, param_raw, param_pred

###  Loading Data Predicted for Direct inverse training

In [3]:
forward_model = MLP(4, 3).to(DEVICE)
forward_model.load_state_dict(torch.load('./models/forward_model_trained.pth',map_location=torch.device('cpu'))['model_state_dict'])
inverse_model = MLP(3, 4).to(DEVICE)
inverse_model.load_state_dict(torch.load('./models/inverse_model_trained.pth',map_location=torch.device('cpu'))['model_state_dict'])


cie_raw, param_raw, cie_pred, param_pred = evaluate_simple_inverse(forward_model, inverse_model, test_loader.dataset)
M = np.shape(param_pred)[0]
mdic = {"param_pred_all": param_pred,"param_test_all": param_raw,"CIE_x_all": cie_raw}
savemat("data_predicted\param_inverse_pred_all.mat",mdic)

B, cie_raw, param_raw, param_pred = struc_remove(cie_raw, param_raw, param_pred)

    
mdic = {"param_pred": param_pred,"param_test": param_raw,"CIE_x": cie_raw, "deleted_row":B}
savemat("data_predicted\param_inverse_pred.mat",mdic)

print(param_pred[1,:])
np.shape(param_pred)

Simple net Design RMSE loss 1.251
Simple net RMSE loss 80.897
Reconstruct RMSE loss 0.538
Reconstruct RMSE loss raw 0.067
43 43
84 83
225 223
337 334
714 710
1375 1370
[118.84330703 254.39965803 588.35666868 147.97332632]


(1405, 4)

### Loading Data predicted for Tandem (fixed decoder)

In [4]:
tandem_model = TandemNet(forward_model, inverse_model)
tandem_model.load_state_dict(torch.load('./models/tandem_net_trained.pth',map_location=torch.device('cpu'))['model_state_dict'])
cie_raw, param_raw, cie_pred, param_pred = evaluate_tandem_accuracy(tandem_model, test_loader.dataset)
mdic = {"param_pred_all": param_pred,"param_test_all": param_raw,"CIE_x_all": cie_raw}
savemat("data_predicted\param_tandem_pred_all.mat",mdic)

B, cie_raw, param_raw, param_pred = struc_remove(cie_raw, param_raw, param_pred)
mdic = {"param_pred": param_pred,"param_test": param_raw,"CIE_x": cie_raw, "deleted_row":B}
savemat("data_predicted\param_tandem_pred.mat",mdic)

print(param_pred[1,:])
np.shape(param_pred)

Tandem net Design RMSE loss 2.042
Tandem Design RMSE loss 138.686
Reconstruct RMSE loss 0.149
Reconstruct RMSE loss raw 0.019
6 6
67 66
74 72
83 80
140 136
194 189
244 238
247 240
262 254
266 257
273 263
290 279
337 325
365 352
380 366
403 388
408 392
423 406
441 423
462 443
486 466
511 490
519 497
520 497
587 563
607 582
644 618
688 661
693 665
750 721
780 750
784 753
786 754
808 775
810 776
843 808
846 810
932 895
935 897
972 933
1019 979
1059 1018
1071 1029
1079 1036
1098 1054
1100 1055
1130 1084
1131 1084
1146 1098
1147 1098
1166 1116
1196 1145
1198 1146
1327 1274
1330 1276
1351 1296
1362 1306
[126.26355856 250.77042705 618.41008658 146.16572224]


(1354, 4)

### Loading VAE

In [5]:
configs = get_configs('vae')
vae_model = cVAE(configs['input_dim'], configs['latent_dim']).to(DEVICE)
vae_model.load_state_dict(torch.load('./models/vae_trained.pth',map_location=torch.device('cpu'))['model_state_dict'])

param_pred = np.zeros([M, 4*5])

for i in range(5):
    cie_raw, param_raw, cie_pred, param_pred[:,(4*i):(4*i+4)] = evaluate_vae_inverse(forward_model, vae_model, configs, test_loader.dataset)
mdic = {"param_pred_all": param_pred,"param_test_all": param_raw,"CIE_x_all": cie_raw}
savemat("data_predicted\param_vae_pred_all.mat",mdic)

B, cie_raw, param_raw, param_pred = struc_remove(cie_raw, param_raw, param_pred)
    
# Saving the predicted data
mdic = {"param_pred": param_pred,"param_test": param_raw,"CIE_x": cie_raw, "deleted_row":B}
savemat("data_predicted\param_vae_pred.mat",mdic)

print(param_pred[1,:])
np.shape(param_pred)

Simple net Design RMSE loss 1.607
Simple net RMSE loss 106.041
Reconstruct RMSE loss 0.736
Reconstruct RMSE loss raw 0.093
Simple net Design RMSE loss 1.600
Simple net RMSE loss 104.775
Reconstruct RMSE loss 0.720
Reconstruct RMSE loss raw 0.091
Simple net Design RMSE loss 1.602
Simple net RMSE loss 105.146
Reconstruct RMSE loss 0.737
Reconstruct RMSE loss raw 0.093
Simple net Design RMSE loss 1.620
Simple net RMSE loss 106.870
Reconstruct RMSE loss 0.744
Reconstruct RMSE loss raw 0.094
Simple net Design RMSE loss 1.583
Simple net RMSE loss 103.526
Reconstruct RMSE loss 0.718
Reconstruct RMSE loss raw 0.091
43 43
84 83
107 105
225 222
598 594
616 611
677 671
714 707
805 797
923 914
970 960
1055 1044
1059 1047
1242 1229
1256 1242
1295 1280
1304 1288
1327 1310
1372 1354
1377 1358
[145.05196301 282.23942966 528.59092711 134.18699505 127.75857004
 252.55885986 581.54104267 142.26505939 109.73970427 244.17204076
 570.75013439 150.42855778 121.97557427 254.82170876 569.81881727
 145.20020863

(1391, 20)

### Loading GAN


In [6]:
configs = get_configs('gan')
cgan = cGAN(3, 4, noise_dim = configs['noise_dim'], hidden_dim = 128).to(DEVICE)
cgan.load_state_dict(torch.load('./models/gan_trained.pth',map_location=torch.device('cpu'))['model_state_dict'])


param_pred = np.zeros([M, 4*5])
for i in range(5):
    cie_raw, param_raw, cie_pred, param_pred[:,(4*i):(4*i+4)] = evaluate_gan_inverse(forward_model, cgan, configs, test_loader.dataset)
mdic = {"param_pred_all": param_pred,"param_test_all": param_raw,"CIE_x_all": cie_raw}
savemat("data_predicted\param_gan_pred_all.mat",mdic)

B, cie_raw, param_raw, param_pred = struc_remove(cie_raw, param_raw, param_pred)

# Saving the predicted data
mdic = {"param_pred": param_pred,"param_test": param_raw,"CIE_x": cie_raw, "deleted_row":B}
savemat("data_predicted\param_gan_pred.mat",mdic)

print(param_pred[1,:])
np.shape(param_pred)

Simple net Design RMSE loss 2.790
Simple net RMSE loss 183.772
Reconstruct RMSE loss 1.584
Reconstruct RMSE loss raw 0.195
Simple net Design RMSE loss 2.834
Simple net RMSE loss 187.137
Reconstruct RMSE loss 1.598
Reconstruct RMSE loss raw 0.196
Simple net Design RMSE loss 2.801
Simple net RMSE loss 185.508
Reconstruct RMSE loss 1.584
Reconstruct RMSE loss raw 0.195
Simple net Design RMSE loss 2.776
Simple net RMSE loss 181.064
Reconstruct RMSE loss 1.599
Reconstruct RMSE loss raw 0.196
Simple net Design RMSE loss 2.814
Simple net RMSE loss 186.247
Reconstruct RMSE loss 1.589
Reconstruct RMSE loss raw 0.194
6 6
9 8
16 14
21 18
22 18
37 32
48 42
52 45
59 51
82 73
110 100
112 101
113 101
119 106
133 119
140 125
142 126
143 126
177 159
184 165
185 165
189 168
191 169
210 187
213 189
219 194
221 195
227 200
241 213
256 227
262 232
276 245
286 254
296 263
306 272
310 275
314 278
317 280
323 285
325 286
326 286
328 287
342 300
349 306
352 308
353 308
359 313
363 316
375 327
379 330
380 330
3

(1221, 20)

### Loading INN

In [7]:
configs = get_configs('inn')
model = INN(configs['ndim_total'], configs['input_dim'], configs['output_dim'], dim_z = configs['latent_dim']).to(DEVICE)
model.load_state_dict(torch.load('./models/inn_trained.pth',map_location=torch.device('cpu'))['model_state_dict'], strict=False)

param_pred = np.zeros([M, 4*5])
for i in range(5):
    cie_raw, param_raw, cie_pred, param_pred[:,(4*i):(4*i+4)] = evaluate_inn_inverse(forward_model, model, configs, test_loader.dataset)
mdic = {"param_pred_all": param_pred,"param_test_all": param_raw,"CIE_x_all": cie_raw}
savemat("data_predicted\param_inn_pred_all.mat",mdic)


B, cie_raw, param_raw, param_pred = struc_remove(cie_raw, param_raw, param_pred)

# Saving the predicted data
mdic = {"param_pred": param_pred,"param_test": param_raw,"CIE_x": cie_raw, "deleted_row":B}
savemat("data_predicted\param_inn_pred.mat",mdic)

print(param_pred[1,:])

np.shape(param_pred)

Simple net Design RMSE loss 1.797
Simple net RMSE loss 114.028
Reconstruct RMSE loss 0.554
Reconstruct RMSE loss raw 0.069
Simple net Design RMSE loss 1.903
Simple net RMSE loss 125.362
Reconstruct RMSE loss 0.603
Reconstruct RMSE loss raw 0.076
Simple net Design RMSE loss 1.958
Simple net RMSE loss 131.435
Reconstruct RMSE loss 0.609
Reconstruct RMSE loss raw 0.076
Simple net Design RMSE loss 1.972
Simple net RMSE loss 131.849
Reconstruct RMSE loss 0.632
Reconstruct RMSE loss raw 0.079
Simple net Design RMSE loss 2.025
Simple net RMSE loss 136.554
Reconstruct RMSE loss 0.657
Reconstruct RMSE loss raw 0.082
11 11
13 12
36 34
53 50
56 52
57 52
121 115
123 116
145 137
182 173
197 187
198 187
201 189
204 191
208 194
225 210
247 231
251 234
259 241
295 276
318 298
337 316
381 359
414 391
424 400
449 424
474 448
480 453
517 489
527 498
531 501
542 511
546 514
556 523
558 524
601 566
613 577
616 579
632 594
661 622
729 689
740 699
761 719
765 722
771 727
777 732
780 734
781 734
787 739
795 7

(1320, 20)